In [1]:
import os
import re
import json

def get_files():
    files = []
    for dirpath, dirnames, filenames in os.walk('public'):
        for filename in filenames:
            if filename.endswith('.md') or filename.endswith('.pdf'):
                if not filename.endswith('homepage.md'):
                    files.append(os.path.join(dirpath, filename))

    return files


In [2]:
md_files = get_files()

In [3]:
md_files[:3]

['public/docs/development/test2.md',
 'public/docs/development/test.md',
 'public/docs/projects/Capstone-group16-melbourne-pollen/README.md']

In [4]:
def modify_md_file(md_file_path):
    # 读取文件内容
    with open(md_file_path, 'r') as f:
        content = f.read()

    # 使用正则表达式提取图片路径
    pattern = r'!\[.*?\]\((.*?)\)'
    matches = re.findall(pattern, content)

    # 遍历所有匹配的图片路径
    for match in matches:
        # 如果路径不以/docs/开头，则替换为以/docs/开头的路径
        if not match.startswith('/docs/'):
            # 获取文件名和文件夹名
            filename = os.path.basename(match)
            dirname = os.path.dirname(match)
            print(dirname)

            # 构造新的路径
            starts = '/' + '/'.join(md_file_path.split('/')[1:]) + '/'
            new_path = starts + os.path.join(dirname, filename)

            # 替换旧路径为新路径
            content = content.replace(match, new_path)

    # 写入修改后的文件内容
    with open(md_file_path, 'w') as f:
        f.write(content)

for path in md_files:
    if '.md' in path:
        modify_md_file(path)


In [5]:
with open('doc_config.json') as f:
    info = json.load(f)

def get_info(desc):
    for item in info:
        if desc in item:
            return item[desc]['name'], item[desc]['title']
    info.append(
        {
            desc: {
                'name': desc,
                'title': desc
            }
        }
    )
    return desc, desc

In [6]:
data = []

docs_dir = 'public/docs/'

id = 0

for md_file in md_files:
    # remove the 'public/' prefix and split the path
    md_file_path = md_file[len('public/'):]
    md_file_parts = md_file_path.split('/')

    # initialize variables for traversing the data structure
    current_level = data
    current_path = docs_dir

    # traverse the data structure to find the correct location to add the new item
    for md_file_part in md_file_parts[:-1]:
        current_path = os.path.join(current_path, md_file_part)
        found_item = None
        for item in current_level:
            if item.get('desc') == md_file_part:
                found_item = item
                break
        if found_item is None:
            name, title = get_info(md_file_part)
            new_item = {'name': name,
                        'id': id,
                        'desc': md_file_part,
                        'children': []}
            id += 1
            current_level.append(new_item)
            current_level = new_item['children']
        else:
            current_level = found_item['children']

    # add the new item to the data structure
    filename = md_file_parts[-1]
    name, title = get_info(filename)
    new_item = {'name': name,
                'id': id,
                'desc': filename,
                'path': md_file_path,
                'title': title}
    id += 1
    current_level.append(new_item)

# print(data)


In [7]:
# context = json.dumps(info, indent=2)

with open('doc_config.json', 'w') as f:
    json.dump(info, f, indent=2)


# data

[{'name': 'docs',
  'id': 0,
  'desc': 'docs',
  'children': [{'name': 'development',
    'id': 1,
    'desc': 'development',
    'children': [{'name': 'test2.md',
      'id': 2,
      'desc': 'test2.md',
      'path': 'docs/development/test2.md',
      'title': 'test2.md'},
     {'name': 'test.md',
      'id': 3,
      'desc': 'test.md',
      'path': 'docs/development/test.md',
      'title': 'test.md'}]},
   {'name': 'projects',
    'id': 4,
    'desc': 'projects',
    'children': [{'name': 'Capstone-group16-melbourne-pollen',
      'id': 5,
      'desc': 'Capstone-group16-melbourne-pollen',
      'children': [{'name': 'README.md',
        'id': 6,
        'desc': 'README.md',
        'path': 'docs/projects/Capstone-group16-melbourne-pollen/README.md',
        'title': 'README.md'},
       {'name': 'feature_analysis',
        'id': 7,
        'desc': 'feature_analysis',
        'children': [{'name': 'feature_analysis_report.md',
          'id': 8,
          'desc': 'feature_analysis

In [13]:
new_data = data[0]['children']

order = ['notes', 'projects', 'development']

new_data = [item for item in new_data]
new_data = sorted(new_data, key=lambda x:order.index(x['desc']))

context = "const data = " + json.dumps(new_data, indent=2) + ";\nexport default data;"

with open('src/views/documents/plugin/doc.js', 'w') as f:
    f.write(context)


In [17]:
import re

# 定义正则表达式
pattern = r'(?<!\$)\$(?!\$)'

# 测试字符串
test_str = 'This is a $$test string with $$single $dollar$ signs'

# 替换字符串
result = re.sub(pattern, '$$', test_str)

print(result) # 输出 'This is a $$test string with $$single $$dollar$$ signs'


This is a $$test string with $$single $$dollar$$ signs


In [49]:
import os
import re
import json

# 定义图片文件夹路径
img_path = "src/assets/img"

# 定义正则表达式匹配文件名中的信息
pattern = re.compile(r"(.+)_@(.+)\.")

# 遍历digital_arts文件夹中的所有图片
digital_arts = []
digital_arts_path = os.path.join(img_path, "digit_arts")
for filename in os.listdir(digital_arts_path):
    if filename == ".DS_Store":
        continue
    # print(filename)
    # 匹配文件名中的信息
    match = pattern.match(filename)
    # print(match)
    if match:
        img_path = os.path.join("assets", "img", "digit_arts", filename)
        title = match.group(1)
        author = "@" + match.group(2)
        digital_arts.append({
            "img": f'require("{img_path}")',
            # "img": img_path,
            "title": title,
            "author": author,
            "type": "da"
        })
    else:
        img_path = os.path.join("assets", "img", "digit_arts", filename)
        title = filename.split('.')[0]
        digital_arts.append({
            "img": f'require("{img_path}")',
            # "img": img_path,
            "title": title,
            "type": "da"
        })

# 将digital_arts保存到digital.js文件中
content = "const DigitalArts = " + json.dumps(digital_arts, indent=2) + ";\nexport default DigitalArts;"
with open("src/views/documents/info/digital_arts.js", "w") as f:
    f.write(content)
    

# 遍历bg_collection文件夹中的所有图片

img_path = "src/assets/img"
gallery_bg = []
gallery_bg_path = os.path.join(img_path, "bg_collection")
for filename in os.listdir(gallery_bg_path):
    if filename == ".DS_Store":
        continue
    # 匹配文件名中的信息
    match = pattern.match(filename)
    if match:
        img_path = os.path.join("assets", "img", "bg_collection", filename)
        title = match.group(1)
        author = "@" + match.group(2) if match.group(2) else ""
        gallery_bg.append({
            "img": f'require("{img_path}")',
            # "img": img_path,
            "title": title,
            "author": author,
            "type": "bg"
        })
    else:
        img_path = os.path.join("assets", "img", "bg_collection", filename)
        title = filename.split('.')[0]
        gallery_bg.append({
            "img": f'require("{img_path}")',
            # "img": img_path,
            "title": title,
            "type": "bg"
        })

# 将gallery_bg保存到gallery_bg.js文件中
content = "const GalleryBg = " + json.dumps(gallery_bg, indent=2) + ";\nexport default GalleryBg;"
with open("src/views/documents/info/gallery_bg.js", "w") as f:
    f.write(content)
# print(gallery_bg)


src_pattern = r'\\\"(.*)\\\"'

# 读取文件并替换每一行中匹配到的字符串
with open('src/views/documents/info/digital_arts.js', 'r') as f:
    lines = f.readlines()

with open('src/views/documents/info/digital_arts.js', 'w') as f:
    for line in lines:
        src = re.findall(src_pattern, line)
        if src:
            f.write(f"    \"img\": require(\"{src[0]}\"),\n")
        else:
            f.write(line)


# 读取文件并替换每一行中匹配到的字符串
with open('src/views/documents/info/gallery_bg.js', 'r') as f:
    lines = f.readlines()

with open('src/views/documents/info/gallery_bg.js', 'w') as f:
    for line in lines:
        src = re.findall(src_pattern, line)
        if src:
            f.write(f"    \"img\": require(\"{src[0]}\"),\n")
        else:
            f.write(line)